# Exercise 4

Due: Tue November 19, 8:00am


## Node2Vec

1. Implement custom dataset that samples pq-walks
   - Use the utility function from torch_cluster that actually performs the walks
2. Implement Node2Vec module and training
   - Node2Vec essentially consists of a torch.Embedding module and a loss function
3. Evaluate node classification performance on Cora
4. Evaluate on Link Prediction: Cora, PPI
   - use different ways to combine the node two embeddings for link prediction

Bonus Question: are the predictions stable wrt to the random seeds of the walks?


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
print(torch.__version__)
!pip install torch-scatter torch-sparse torch-cluster torch-geometric -f https://data.pyg.org/whl/torch-{torch.__version__}.html

2.5.1+cpu
Looking in links: https://data.pyg.org/whl/torch-2.5.1+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.0/544.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 785.3/785.3 kB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.9/241.9 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.9/208.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.2/319.2 kB 19.6 MB/s eta 0:00:00


In [3]:
import torch
import torch_geometric as pyg
from tqdm import tqdm
import torch_cluster
import sklearn
from sklearn.metrics import accuracy_score

from typing import Optional
import numpy as np
import os

In [4]:
# find device
if torch.cuda.is_available():  # NVIDIA
    device = torch.device("cuda")
elif torch.backends.mps.is_available():  # apple M1/M2
    device = torch.device("mps")
else:
    device = torch.device("cpu")
device

device(type='cpu')

In [5]:
cora_dataset = pyg.datasets.Planetoid(root="./dataset/cora", name="Cora")
cora = cora_dataset[0]
ppi_dataset = pyg.datasets.PPI(root="./dataset/ppi")
ppi = ppi_dataset[0]

Processing...
Done!
Extracting dataset/ppi/ppi.zip
Processing...
/usr/local/lib/python3.10/dist-packages/networkx/readwrite/json_graph/node_link.py:287: FutureWarning: 
The default value will be changed to `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_graph(data, edges="links") to preserve current behavior, or
  nx.node_link_graph(data, edges="edges") for forward compatibility.
  warnings.warn(
Done!


In [6]:
cora

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [7]:
ppi

Data(x=[1767, 50], edge_index=[2, 32318], y=[1767, 121])

In [8]:
seed = 0

In [9]:
def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

In [10]:
set_seed(seed)

Random seed set as 0


## node2vec embedding training

Here the main training and everything on the graph level is happening.

It might be a good idea to create a dataset of walks (fixed for the whole training process) first to get the whole training process running before attempting to create a train_loader that on-demand samples those walks on-demand.


In [11]:
class PQWalkDataset(torch.utils.data.Dataset):
    def __init__(
        self,
        data,
        walk_length,
        walks_per_node=1,
        p=1,
        q=1,
        num_negative_samples=1,
    ):
        self.data = data
        # check if edge_label_index is present
        if hasattr(self.data, "edge_label_index"):
            self.edge_index = self.data.edge_label_index
        else:
            self.edge_index = self.data.edge_index
        self.walk_length = walk_length - 1
        self.walks_per_node = walks_per_node
        self.num_nodes = self.data.num_nodes
        self.p = p
        self.q = q
        self.num_negative_samples = num_negative_samples

        self._start_nodes = torch.arange(self.num_nodes).repeat(self.walks_per_node)
        self._negative_start_nodes = torch.arange(self.num_nodes).repeat(
            self.walks_per_node * self.num_negative_samples
        )
        self._pos_samples = self._get_pos_samples()
        self._neg_samples = self._get_neg_samples()

    def _get_pos_samples(self):
        return torch_cluster.random_walk(
            self.edge_index[0],
            self.edge_index[1],
            start=self._start_nodes,
            walk_length=self.walk_length,
            p=self.p,
            q=self.q,
        )

    def _get_neg_samples(self):
        negative_samples = torch.randint(
            0, self.num_nodes, (self._negative_start_nodes.shape[0], self.walk_length)
        )
        negative_samples = torch.cat(
            [self._negative_start_nodes.view(-1, 1), negative_samples], dim=-1
        )
        return negative_samples

    def __len__(self):
        return len(self._pos_samples)

    def __getitem__(self, idx):
        walk = self._pos_samples[idx]
        neg_sample = self._neg_samples[idx]
        return walk, neg_sample

In [12]:
class PQWalkIterableDataset(torch.utils.data.IterableDataset):
    def __init__(
        self,
        data,
        walk_length=10,
        walks_per_node=10,
        p=1,
        q=1,
        num_negative_samples=1,
    ):
        self.data = data
        # check if edge_label_index is present
        if hasattr(self.data, "edge_label_index"):
            self.edge_index = self.data.edge_label_index
        else:
            self.edge_index = self.data.edge_index
        self.walk_length = walk_length - 1
        self.walks_per_node = walks_per_node
        self.num_nodes = self.data.num_nodes
        self.p = p
        self.q = q
        self.num_negative_samples = num_negative_samples

    def _generate_negative_samples(self, start_nodes):
        # Repeat nodes for each negative sample
        nodes = start_nodes.repeat(self.num_negative_samples)

        # Generate random walks for negative samples
        rw = torch.randint(
            self.num_nodes,
            (nodes.size(0), self.walk_length),
            dtype=nodes.dtype,
            device=nodes.device,
        )
        # Concatenate start nodes with random walks
        rw = torch.cat([nodes.view(-1, 1), rw], dim=-1)
        return rw

    def __iter__(self):
        worker_info = torch.utils.data.get_worker_info()
        worker_id = 0 if worker_info is None else worker_info.id
        num_workers = 1 if worker_info is None else worker_info.num_workers

        # Calculate nodes per worker
        nodes_per_worker = self.num_nodes // num_workers
        start_node = worker_id * nodes_per_worker
        end_node = (
            start_node + nodes_per_worker
            if worker_id < num_workers - 1
            else self.num_nodes
        )
        # worker_nodes = end_node - start_node

        # Generate start nodes array that ensures walks_per_node samples for each node
        start_nodes = torch.arange(start_node, end_node).repeat_interleave(
            self.walks_per_node
        )
        total_walks = len(start_nodes)

        # Shuffle all start nodes
        perm = torch.randperm(total_walks)
        start_nodes = start_nodes[perm]

        walks = torch_cluster.random_walk(
            self.edge_index[0],
            self.edge_index[1],
            start=start_nodes,
            walk_length=self.walk_length,
            p=self.p,
            q=self.q,
        )
        neg_samples = self._generate_negative_samples(start_nodes)

        for walk, neg_sample in zip(walks, neg_samples):
            yield walk, neg_sample

In [13]:
class Node2Vec(torch.nn.Module):
    def __init__(self, embedding_dim: int, num_nodes: int):
        super(Node2Vec, self).__init__()
        self.embedding_dim = embedding_dim
        self.num_nodes = num_nodes
        self.embedding = torch.nn.Embedding(num_nodes, embedding_dim)
        self._EPS = 1e-15

    def loss(self, pos_sample, neg_sample):
        assert torch.equal(pos_sample[:, 0], neg_sample[:, 0])
        start_nodes = pos_sample[:, 0]
        pos_sample_rest = pos_sample[:, 1:].contiguous()
        neg_sample_rest = neg_sample[:, 1:].contiguous()

        h_start = self.embedding(start_nodes).view(
            pos_sample.shape[0], 1, self.embedding_dim
        )
        h_rest = self.embedding(pos_sample_rest).view(
            pos_sample.shape[0], -1, self.embedding_dim
        )

        out = (h_start * h_rest).sum(dim=-1).view(-1)
        pos_loss = -torch.log(torch.sigmoid(out) + self._EPS).mean()

        h_start = self.embedding(start_nodes).view(
            neg_sample.shape[0], 1, self.embedding_dim
        )
        h_rest = self.embedding(neg_sample_rest.view(-1)).view(
            neg_sample.shape[0], -1, self.embedding_dim
        )

        out = (h_start * h_rest).sum(dim=-1).view(-1)
        neg_loss = -torch.log(1 - torch.sigmoid(out) + self._EPS).mean()

        return pos_loss + neg_loss

    def get_embedding(self):
        return self.embedding.weight

    def forward(self, pos_sample, neg_sample):
        return self.loss(pos_sample, neg_sample)

In [ ]:
def train_node2vec(
    data: pyg.data.Data,
    walk_length: int,
    walks_per_node: int,
    embedding_dim: int,
    p: float = 1,
    q: float = 1,
    num_negative_samples: int = 1,
    batch_size: int = 32,
    lr: float = 0.01,
    num_epochs: int = 200,
    num_workers: int = 4,
):
    dataset = PQWalkIterableDataset(
        data=data,
        walk_length=walk_length,
        walks_per_node=walks_per_node,
        num_negative_samples=num_negative_samples,
        p=p,
        q=q,
    )

    dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=True if device.type == "cuda" else False,
    )

    model = Node2Vec(embedding_dim, data.num_nodes)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer,
        T_max=num_epochs,  # Full period of cosine annealing
        eta_min=1e-5,  # Minimum learning rate
    )
    model = model.to(device)
    model.train()

    for epoch in range(num_epochs):
        total_loss = 0
        num_batches = 0

        for pos_sample, neg_sample in dataloader:
            pos_sample = pos_sample.to(device)
            neg_sample = neg_sample.to(device)
            optimizer.zero_grad()
            loss = model(pos_sample, neg_sample)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            num_batches += 1

        # Calculate average loss for this epoch
        avg_loss = total_loss / num_batches
        scheduler.step()
        if (epoch + 1) % 10 == 0:
            current_lr = optimizer.param_groups[0]["lr"]
            print(f"Epoch {epoch+1:02d}, Loss: {avg_loss:.4f}, LR: {current_lr:.6f}")

    return model.get_embedding()

In [15]:
cora_embeddings = train_node2vec(
    cora, 100, 10, 128, p=0.5, q=2.0, lr=0.01, num_epochs=100
)

Epoch 10, Loss: 1.1265, LR: 0.009756
Epoch 20, Loss: 1.1144, LR: 0.009046
Epoch 30, Loss: 1.1078, LR: 0.007941
Epoch 40, Loss: 1.0961, LR: 0.006549
Epoch 50, Loss: 1.0871, LR: 0.005005
Epoch 60, Loss: 1.0760, LR: 0.003461
Epoch 70, Loss: 1.0691, LR: 0.002069
Epoch 80, Loss: 1.0629, LR: 0.000964
Epoch 100, Loss: 1.0583, LR: 0.000010


In [16]:
embedding_dim = cora_embeddings.shape[1]

## Node classification performance

just a small MLP or even linear layer on the embeddings to predict node classes. Accuracy should be above 60%. Please compare your results to those you achieved with GNNs.


In [17]:
# as the simple MLP is pretty straightforward
model = torch.nn.Sequential(
    torch.nn.Linear(embedding_dim, 256),  # Input layer
    torch.nn.ReLU(),
    torch.nn.Linear(256, 128),  # Hidden layer 2
    torch.nn.ReLU(),
    torch.nn.Linear(128, cora_dataset.num_classes),  # Output layer
)
model = model.to(device)

In [18]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)  # define an optimizer
criterion = torch.nn.CrossEntropyLoss()  # define loss function

cora_embeddings = cora_embeddings.to(device)
cora = cora.to(device)

for epoch in range(100):  # 100 epochs
    model.train()
    optimizer.zero_grad()
    out = model(cora_embeddings[cora.train_mask])  # forward pass
    loss = criterion(out, cora.y[cora.train_mask])
    loss.backward()
    optimizer.step()

    # print out loss info
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.3e}")


def get_accuracy(model, embeddings, y, mask):
    out = model(embeddings[mask])
    pred = out.argmax(dim=1)
    acc = accuracy_score(y[mask].cpu().numpy(), pred.cpu().detach().numpy())
    return acc


train_acc = get_accuracy(model, cora_embeddings, cora.y, cora.train_mask)
val_acc = get_accuracy(model, cora_embeddings, cora.y, cora.val_mask)
test_acc = get_accuracy(model, cora_embeddings, cora.y, cora.test_mask)

print(
    f"node classification accuracy for cora: {test_acc:.2f} (train: {train_acc:.2f}, val: {val_acc:.2f})"
)

Epoch 10, Loss: 2.090e-02
Epoch 20, Loss: 1.767e-05
Epoch 30, Loss: 3.293e-06
Epoch 40, Loss: 4.402e-07
Epoch 50, Loss: 1.584e-07
Epoch 60, Loss: 1.005e-07
Epoch 70, Loss: 8.174e-08
Epoch 80, Loss: 7.408e-08
Epoch 90, Loss: 6.897e-08
Epoch 100, Loss: 6.557e-08
node classification accuracy for cora: 0.67 (train: 1.00, val: 0.67)


## link prediction on trained embeddings

this should only train simple MLPs.

Note: for link prediction to be worthwhile, one needs to train the embeddings on a subset of the graph (less edges, same nodes) instead of the whole graph.


In [19]:
# for link prediction, do something like the following
link_splitter = pyg.transforms.RandomLinkSplit(is_undirected=True)
train_data, val_data, test_data = link_splitter(cora)
train_data
# the positive and negative edges are in "edge_label_index" with "edge_label"
# indicating whether an edge is a true edge or not.

Data(x=[2708, 1433], edge_index=[2, 7392], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[7392], edge_label_index=[2, 7392])

In [20]:
test_data

Data(x=[2708, 1433], edge_index=[2, 8446], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[2110], edge_label_index=[2, 2110])

In [21]:
# retrain node2vec on train_data

In [ ]:
def calculate_mrr(embeddings, pos_edge_index, all_edges, mode="filtered", k=None):
    """
    Calculate MRR with different filtering modes

    mode: 'raw', 'filtered', or 'more-filtered'
    k: if not None, only consider the top k ranks
    """
    mrr_list = []
    num_nodes = embeddings.size(0)

    # Convert existing edges to set for faster lookup
    existing_edges = set(map(tuple, all_edges.t().tolist()))

    with torch.no_grad():
        for i in range(pos_edge_index.size(1)):
            source = pos_edge_index[0, i]
            target = pos_edge_index[1, i]

            source_emb = embeddings[source].unsqueeze(0)
            all_scores = torch.mm(source_emb, embeddings.t()).squeeze()

            # Different filtering modes
            if mode == "raw":
                # No filtering - includes all edges (not recommended)
                pass

            elif mode == "filtered":
                # Filter out existing edges except the target
                for j in range(num_nodes):
                    if (source.item(), j) in existing_edges and j != target.item():
                        all_scores[j] = float("-inf")

            elif mode == "more-filtered":
                # Filter existing edges and self-loops
                for j in range(num_nodes):
                    if (
                        (source.item(), j) in existing_edges and j != target.item()
                    ) or j == source.item():
                        all_scores[j] = float("-inf")

            sorted_indices = torch.argsort(all_scores, descending=True)
            rank = (sorted_indices == target).nonzero().item() + 1

            if k is not None and rank > k:
                mrr_list.append(0)
            else:
                mrr_list.append(1.0 / rank)

    return sum(mrr_list) / len(mrr_list)


def get_link_labels(pos_edge_index, neg_edge_index):
    """
    Creates labels for positive and negative edges
    """
    num_links = pos_edge_index.size(1) + neg_edge_index.size(1)
    link_labels = torch.zeros(num_links, dtype=torch.float)
    link_labels[: pos_edge_index.size(1)] = 1.0
    return link_labels


def get_edge_embeddings(embeddings, edge_index, merge_method: str = "average"):
    """
    Combine node embeddings to create edge embeddings
    """
    # Get node embeddings for both source and target nodes
    src_embeddings = embeddings[edge_index[0]]
    dst_embeddings = embeddings[edge_index[1]]

    # Different ways to combine the embeddings
    if merge_method == "hadamard":
        edge_embedding = src_embeddings * dst_embeddings
    elif merge_method == "average":
        edge_embedding = (src_embeddings + dst_embeddings) / 2
    elif merge_method == "l1":
        edge_embedding = torch.abs(src_embeddings - dst_embeddings)
    elif merge_method == "l2":
        edge_embedding = (src_embeddings - dst_embeddings) ** 2
    return edge_embedding


def evaluate_link_prediction(
    embeddings,
    edge_classifier,
    pos_edge_index,
    neg_edge_index,
    merge_method: str = "average",
):
    """
    Evaluate link prediction performance
    """
    # Get edge embeddings
    with torch.no_grad():  # Don't track gradients for embeddings
        pos_edge_embeddings = get_edge_embeddings(
            embeddings, pos_edge_index, merge_method
        )
        neg_edge_embeddings = get_edge_embeddings(
            embeddings, neg_edge_index, merge_method
        )

        # Combine positive and negative edge embeddings
        edge_embeddings = torch.cat([pos_edge_embeddings, neg_edge_embeddings], dim=0)

        # Create labels
        labels = get_link_labels(pos_edge_index, neg_edge_index)

    # Evaluate
    edge_classifier.eval()
    with torch.no_grad():
        pred = edge_classifier(edge_embeddings).squeeze()
        auc_score = sklearn.metrics.roc_auc_score(labels.cpu(), pred.cpu())
        ap_score = sklearn.metrics.average_precision_score(labels.cpu(), pred.cpu())
    all_edges = torch.cat([pos_edge_index, neg_edge_index], dim=1)
    mrr = calculate_mrr(embeddings, pos_edge_index, all_edges)
    return auc_score, ap_score, mrr


def train_and_evaluate_link_prediction(
    data,
    walk_length: int = 100,
    walks_per_node: int = 10,
    embedding_dim: int = 128,
    p: float = 1,
    q: float = 1,
    lr: float = 0.01,
    num_epochs: int = 100,
    merge_method: str = "average",
    num_workers: int = 4,
):
    link_splitter = pyg.transforms.RandomLinkSplit(is_undirected=True)
    train_data, val_data, test_data = link_splitter(data)

    embeddings = train_node2vec(
        train_data,
        walk_length,
        walks_per_node,
        embedding_dim,
        p,
        q,
        lr=lr,
        num_epochs=num_epochs,
        num_workers=num_workers,
    )
    embeddings = embeddings.detach()

    # Train a simple classifier
    edge_classifier = torch.nn.Sequential(
        torch.nn.Linear(embeddings.shape[1], 64),
        torch.nn.ReLU(),
        torch.nn.Linear(64, 1),
        torch.nn.Sigmoid(),
    ).to(device)

    # Train the classifier
    optimizer = torch.optim.AdamW(edge_classifier.parameters(), lr=0.01)
    criterion = torch.nn.BCELoss()

    pos_edge_embeddings = get_edge_embeddings(
        embeddings,
        train_data.edge_label_index[:, train_data.edge_label == 1],
        merge_method,
    )
    neg_edge_embeddings = get_edge_embeddings(
        embeddings,
        train_data.edge_label_index[:, train_data.edge_label == 0],
        merge_method,
    )

    # Combine positive and negative edge embeddings
    edge_embeddings = torch.cat([pos_edge_embeddings, neg_edge_embeddings], dim=0)

    # Create labels
    labels = get_link_labels(
        train_data.edge_label_index[:, train_data.edge_label == 1],
        train_data.edge_label_index[:, train_data.edge_label == 0],
    )

    edge_embeddings = edge_embeddings.to(device)
    labels = labels.to(device)

    # Training loop
    edge_classifier.train()
    for _ in range(100):
        optimizer.zero_grad()
        out = edge_classifier(edge_embeddings).squeeze()
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()

    # Evaluate on validation set
    val_auc, val_ap, val_mrr = evaluate_link_prediction(
        embeddings,
        edge_classifier,
        val_data.edge_label_index[:, val_data.edge_label == 1],
        val_data.edge_label_index[:, val_data.edge_label == 0],
        merge_method,
    )

    # Evaluate on test set
    test_auc, test_ap, test_mrr = evaluate_link_prediction(
        embeddings,
        edge_classifier,
        test_data.edge_label_index[:, test_data.edge_label == 1],
        test_data.edge_label_index[:, test_data.edge_label == 0],
        merge_method,
    )

    return embeddings, (val_auc, val_ap, val_mrr), (test_auc, test_ap, test_mrr)

In [23]:
# use those (new) embeddings for link prediction
print("Link prediction on Cora:")
cora_results = train_and_evaluate_link_prediction(
    cora, 100, 10, 128, p=0.5, q=2.0, lr=0.01, num_epochs=50, merge_method="average"
)
cora_embeddings, cora_val_results, cora_test_results = cora_results
print(f"Cora embeddings shape: {cora_embeddings.shape}")
print(
    f"Cora validation results: AUC: {cora_val_results[0]:.4f}, AP: {cora_val_results[1]:.4f}, MRR: {cora_val_results[2]:.4f}"
)
print(
    f"Cora test results: AUC: {cora_test_results[0]:.4f}, AP: {cora_test_results[1]:.4f}, MRR: {cora_test_results[2]:.4f}"
)

Link prediction on Cora:
Epoch 10, Loss: 1.2242, LR: 0.009046
Epoch 20, Loss: 1.1255, LR: 0.006549
Epoch 30, Loss: 1.0587, LR: 0.003461
Epoch 40, Loss: 1.0121, LR: 0.000964
Epoch 50, Loss: 1.0016, LR: 0.000010
Cora embeddings shape: torch.Size([2708, 128])
Cora validation results: AUC: 0.6281, AP: 0.6390, MRR: 0.0127
Cora test results: AUC: 0.6248, AP: 0.6466, MRR: 0.0101


In [24]:
print("Link prediction on PPI:")
ppi_results = train_and_evaluate_link_prediction(
    ppi, 100, 10, 128, p=0.5, q=2.0, lr=0.01, num_epochs=50, merge_method="average"
)
ppi_embeddings, ppi_val_results, ppi_test_results = ppi_results
print(f"PPI embeddings shape: {ppi_embeddings.shape}")
print(
    f"PPI validation results: AUC: {ppi_val_results[0]:.4f}, AP: {ppi_val_results[1]:.4f}, MRR: {ppi_val_results[2]:.4f}"
)
print(
    f"PPI test results: AUC: {ppi_test_results[0]:.4f}, AP: {ppi_test_results[1]:.4f}, MRR: {ppi_test_results[2]:.4f}"
)

Link prediction on PPI:
Epoch 10, Loss: 1.3773, LR: 0.009046
Epoch 20, Loss: 1.3706, LR: 0.006549
Epoch 30, Loss: 1.3634, LR: 0.003461
Epoch 40, Loss: 1.3592, LR: 0.000964
Epoch 50, Loss: 1.3571, LR: 0.000010
PPI embeddings shape: torch.Size([1767, 128])
PPI validation results: AUC: 0.8261, AP: 0.8243, MRR: 0.0344
PPI test results: AUC: 0.8386, AP: 0.8339, MRR: 0.0281


In [25]:
# different merge methods on Cora
print("Link prediction on Cora:")
device = torch.device("cpu")
cora = cora.to(device)

for merge_method in ["average", "hadamard", "l1", "l2"]:
    print("Merge method:", merge_method)
    cora_results = train_and_evaluate_link_prediction(
        cora,
        100,
        10,
        128,
        p=0.5,
        q=2.0,
        lr=0.01,
        num_epochs=50,
        merge_method=merge_method,
    )
    cora_embeddings, cora_val_results, cora_test_results = cora_results
    print(f"Cora embeddings shape: {cora_embeddings.shape}")
    print(
        f"Cora validation results: AUC: {cora_val_results[0]:.4f}, AP: {cora_val_results[1]:.4f}, MRR: {cora_val_results[2]:.4f}"
    )
    print(
        f"Cora test results: AUC: {cora_test_results[0]:.4f}, AP: {cora_test_results[1]:.4f}, MRR: {cora_test_results[2]:.4f}"
    )
    print()

Link prediction on Cora:
Merge method: average
Epoch 10, Loss: 1.2271, LR: 0.009046
Epoch 20, Loss: 1.1106, LR: 0.006549
Epoch 30, Loss: 1.0433, LR: 0.003461
Epoch 40, Loss: 1.0047, LR: 0.000964
Epoch 50, Loss: 0.9892, LR: 0.000010
Cora embeddings shape: torch.Size([2708, 128])
Cora validation results: AUC: 0.6346, AP: 0.6597, MRR: 0.0096
Cora test results: AUC: 0.6259, AP: 0.6477, MRR: 0.0113

Merge method: hadamard
Epoch 10, Loss: 1.2472, LR: 0.009046
Epoch 20, Loss: 1.1346, LR: 0.006549
Epoch 30, Loss: 1.0580, LR: 0.003461
Epoch 40, Loss: 1.0098, LR: 0.000964
Epoch 50, Loss: 1.0015, LR: 0.000010
Cora embeddings shape: torch.Size([2708, 128])
Cora validation results: AUC: 0.4756, AP: 0.4926, MRR: 0.0139
Cora test results: AUC: 0.4506, AP: 0.4603, MRR: 0.0083

Merge method: l1
Epoch 10, Loss: 1.2442, LR: 0.009046
Epoch 20, Loss: 1.1363, LR: 0.006549
Epoch 30, Loss: 1.0629, LR: 0.003461
Epoch 40, Loss: 1.0175, LR: 0.000964
Epoch 50, Loss: 1.0062, LR: 0.000010
Cora embeddings shape: tor

In [26]:
# different merge methods on PPI
print("Link prediction on PPI:")
for merge_method in ["average", "hadamard", "l1", "l2"]:
    print("Merge method:", merge_method)
    ppi_results = train_and_evaluate_link_prediction(
        ppi,
        100,
        10,
        128,
        p=0.5,
        q=2.0,
        lr=0.01,
        num_epochs=50,
        merge_method=merge_method,
        num_workers=0,
    )
    ppi_embeddings, ppi_val_results, ppi_test_results = ppi_results
    print(f"PPI embeddings shape: {ppi_embeddings.shape}")
    print(
        f"PPI validation results: AUC: {ppi_val_results[0]:.4f}, AP: {ppi_val_results[1]:.4f}, MRR: {ppi_val_results[2]:.4f}"
    )
    print(
        f"PPI test results: AUC: {ppi_test_results[0]:.4f}, AP: {ppi_test_results[1]:.4f}, MRR: {ppi_test_results[2]:.4f}"
    )
    print()

Link prediction on PPI:
Merge method: average
Epoch 10, Loss: 1.3693, LR: 0.009046
Epoch 20, Loss: 1.3630, LR: 0.006549
Epoch 30, Loss: 1.3542, LR: 0.003461
Epoch 40, Loss: 1.3485, LR: 0.000964
Epoch 50, Loss: 1.3454, LR: 0.000010
PPI embeddings shape: torch.Size([1767, 128])
PPI validation results: AUC: 0.8397, AP: 0.8385, MRR: 0.0262
PPI test results: AUC: 0.8361, AP: 0.8325, MRR: 0.0255

Merge method: hadamard
Epoch 10, Loss: 1.3746, LR: 0.009046
Epoch 20, Loss: 1.3695, LR: 0.006549
Epoch 30, Loss: 1.3621, LR: 0.003461
Epoch 40, Loss: 1.3575, LR: 0.000964
Epoch 50, Loss: 1.3555, LR: 0.000010
PPI embeddings shape: torch.Size([1767, 128])
PPI validation results: AUC: 0.6517, AP: 0.6749, MRR: 0.0278
PPI test results: AUC: 0.6618, AP: 0.6781, MRR: 0.0282

Merge method: l1
Epoch 10, Loss: 1.3780, LR: 0.009046
Epoch 20, Loss: 1.3732, LR: 0.006549
Epoch 30, Loss: 1.3660, LR: 0.003461
Epoch 40, Loss: 1.3618, LR: 0.000964
Epoch 50, Loss: 1.3606, LR: 0.000010
PPI embeddings shape: torch.Size(